In [1]:
%cd ../

/Users/Matteo/PycharmProjects/TrafficEmu


In [2]:
# If you get a SciPy error when installing Emukit, build it from source:

# git clone https://github.com/amzn/Emukit.git
# cd Emukit
# pip install -r requirements/requirements.txt
# python setup.py develop

In [3]:
import numpy as np
import pickle
import emukit as ek
import GPy

from emukit.model_wrappers import GPyModelWrapper
from emukit.experimental_design.experimental_design_loop import ExperimentalDesignLoop
from emukit.core import ParameterSpace, ContinuousParameter, DiscreteParameter
from emukit.core.initial_designs import RandomDesign, latin_design
from emukit.experimental_design.acquisitions import ModelVariance, IntegratedVarianceReduction
from emukit.core.loop import UserFunctionWrapper
from emukit.bayesian_optimization.acquisitions import ExpectedImprovement
from emukit.bayesian_optimization.loops import BayesianOptimizationLoop
from emukit.core.optimization import GradientAcquisitionOptimizer

from sumo_grid_simulation.grid_simulation import Simulator
import experimental_design.config as config

warning in stationary: failed to import cython module: falling back to numpy
warning in coregionalize: failed to import cython module: falling back to numpy
warning in choleskies: failed to import cython module: falling back to numpy


### Pseudocode

Fitting a gaussian process to a simulator using Emukit takes the following form (taken from L48 lectures):

```
initialize GP with some randomly chosen points
while stopping condition is not met:
    compute candidate point(s) using GP and acquisition funciton (model_variance) -> new point
    evaluate this new point with our simulator/user function -> observation
    update model with new observation -> new GP
```

### Simulator

In [4]:
simulator = Simulator(end_time=300)

### User Function
 
This is the function we want to understand. Namedly, how are CO2 emissions and timeLoss (the time lost due to driving below the ideal speed) affected by the following parameters: 

```
gridSize: the size of the grid network where the simulation is carried out
edgeMaxSpeed: legal speed limit in m/s - this can be exceeded (11.11 == 40km/h)
maxSpeed: the absolute maximum velocity of any vehicle in m/s (55.55 == 200 km/h)
edgeLength: length of the roads between intersections in meters
numLanes: number of lanes per road
accel: The acceleration ability of vehicles in m/s^2.
```

*A complete list of parmeters analysed is discussed in our report.*

In [5]:
def user_function_co2(X):
    """  X = inputs - emukit doesnt pass named args, just an NxM ndarray, N is the number of points to evaluate, M is the number of parameters per each point """
    result = []
    i = 0
    
    print(X)
    print(f'\nUser function time loss called with {X.shape[0]} inputs to simulate')

    
    for gridSize, edgeMaxSpeed, maxSpeed, edgeLength, numLanes, accel in X:
        print(f'\nEvaluating input: {i+1} of {X.shape[0]}\n')
        
        alpha = 0.005
        max_number_of_vehicles = ((gridSize - 1) * gridSize * 2 + 4 * gridSize) * edgeLength / 5
        period = 300/(max_number_of_vehicles * alpha)
        
        s = simulator.simulate(
            gridSize      = int(gridSize),
            edgeMaxSpeed  = edgeMaxSpeed,
            maxSpeed      = maxSpeed,
            edgeLength    = edgeLength,
            numberOfLanes = int(numLanes),
            accel         = accel,
            trips_generator_period = period
        )
        # average co2 / average route duration
        CO2 = s['CO2']/s['num_emissions_samples']
        # average emissions / average route length
        result.append(CO2/(s['duration'] * s['speed']))
        print(f'\nOutput {result[-1]}\n')
        i += 1
        
    # expand dims is essential or the acquition function breaks
    return np.expand_dims(np.array(result), 1)  

### Model Inputs / parameter space

In [6]:
parameter_space = config.get_parameter_space()

### Get init, test, and train points

Train points are used to optimize a GP without experimental design - to verify experimental design works.

In [7]:
design = RandomDesign(parameter_space)
init_X = design.get_samples(20)
init_Y = user_function_co2(init_X)

design = RandomDesign(parameter_space)
test_X = design.get_samples(50)
test_Y = user_function_co2(test_X)

design = RandomDesign(parameter_space)
train_X = design.get_samples(270)
train_Y = user_function_co2(train_X)

[[12.         10.97067626 10.66297579 32.76984853  2.          1.68978239]
 [15.         11.74166159  9.18765904 52.78232558  2.          4.28786226]
 [ 8.         24.88901555 14.19594606 65.00321206  2.          4.6764124 ]
 [16.         21.12345183 13.46970084 69.3838183   3.          2.32643367]
 [13.          8.03551549  7.37990675 33.56461575  3.          1.85199115]
 [ 8.         13.09284196 14.30925458 41.92646896  1.          3.37965384]
 [19.         10.43598529 24.17599548 35.337316    3.          1.84639408]
 [19.         22.57024468  6.92500698 54.69876966  2.          4.1378053 ]
 [ 7.         10.38486387 10.45036434 34.85365727  3.          2.73508283]
 [ 4.         12.25409892 10.04399931 50.97430667  2.          3.90339997]
 [14.         17.58002658 17.66383877 65.88173144  1.          2.83955822]
 [13.         24.79226435 16.03047271 60.04023432  1.          3.69740856]
 [12.         20.15363626 11.54139185 33.33965709  2.          4.74028875]
 [17.          8.56931825

 Retrying in 1 seconds

Output 4.2951482130922525


Evaluating input: 18 of 50

 Retrying in 1 seconds

Output 8.470591520975114


Evaluating input: 19 of 50

 Retrying in 1 seconds

Output 1.813289205758099


Evaluating input: 20 of 50

 Retrying in 1 seconds

Output 4.707193245663579


Evaluating input: 21 of 50

 Retrying in 1 seconds

Output 3.163775692447066


Evaluating input: 22 of 50

 Retrying in 1 seconds

Output 6.783134360028919


Evaluating input: 23 of 50

 Retrying in 1 seconds

Output 5.599104743315149


Evaluating input: 24 of 50

 Retrying in 1 seconds

Output 7.95911374985783


Evaluating input: 25 of 50

 Retrying in 1 seconds

Output 2.344685469649831


Evaluating input: 26 of 50

 Retrying in 1 seconds

Output 6.786506007209826


Evaluating input: 27 of 50

 Retrying in 1 seconds

Output 12.963052990183416


Evaluating input: 28 of 50

 Retrying in 1 seconds

Output 10.556876873534991


Evaluating input: 29 of 50

 Retrying in 1 seconds

Output 3.1454306598603283



Output 6.600723612718396


Evaluating input: 64 of 270

 Retrying in 1 seconds

Output 7.074849351319224


Evaluating input: 65 of 270

 Retrying in 1 seconds

Output 5.579303328114911


Evaluating input: 66 of 270

 Retrying in 1 seconds

Output 4.202761139348878


Evaluating input: 67 of 270

 Retrying in 1 seconds

Output 6.429546358181239


Evaluating input: 68 of 270

 Retrying in 1 seconds

Output 6.571458231877725


Evaluating input: 69 of 270

 Retrying in 1 seconds

Output 9.916727940758694


Evaluating input: 70 of 270

 Retrying in 1 seconds

Output 5.1997289345979425


Evaluating input: 71 of 270

 Retrying in 1 seconds

Output 14.904280938104712


Evaluating input: 72 of 270

 Retrying in 1 seconds

Output 12.406346427108751


Evaluating input: 73 of 270

 Retrying in 1 seconds

Output 6.594746617970638


Evaluating input: 74 of 270

 Retrying in 1 seconds

Output 6.049108838574112


Evaluating input: 75 of 270

 Retrying in 1 seconds

Output 12.797393978710119


Evaluati

 Retrying in 1 seconds

Output 3.4047818250021624


Evaluating input: 166 of 270

 Retrying in 1 seconds

Output 8.741845017575542


Evaluating input: 167 of 270

 Retrying in 1 seconds

Output 3.8012228034706768


Evaluating input: 168 of 270

 Retrying in 1 seconds

Output 7.640495369559434


Evaluating input: 169 of 270

 Retrying in 1 seconds

Output 8.394081294955924


Evaluating input: 170 of 270

 Retrying in 1 seconds

Output 9.568340195250473


Evaluating input: 171 of 270

 Retrying in 1 seconds

Output 13.369431322647149


Evaluating input: 172 of 270

 Retrying in 1 seconds

Output 3.083234027078913


Evaluating input: 173 of 270

 Retrying in 1 seconds

Output 3.3956673901592636


Evaluating input: 174 of 270

 Retrying in 1 seconds

Output 4.816279520134806


Evaluating input: 175 of 270

 Retrying in 1 seconds

Output 5.865328763904913


Evaluating input: 176 of 270

 Retrying in 1 seconds

Output 15.90103653024326


Evaluating input: 177 of 270

 Retrying in 1 seconds



 Retrying in 1 seconds

Output 10.609732982562333


Evaluating input: 267 of 270

 Retrying in 1 seconds

Output 5.802694860491142


Evaluating input: 268 of 270

 Retrying in 1 seconds

Output 5.775531280246777


Evaluating input: 269 of 270

 Retrying in 1 seconds

Output 3.974065216666643


Evaluating input: 270 of 270

 Retrying in 1 seconds

Output 4.052731224274325



### Experimental Design - Model Variance

#### Model / Emulator (GP)

Our surrogate model is our emulator. In this case, a gaussian process.

In [8]:
emulator_ed = GPy.models.GPRegression(init_X, init_Y)
emukit_model_ed = GPyModelWrapper(emulator_ed, n_restarts=5)
emukit_model_ed.optimize()
emulator_ed

Optimization restart 1/5, f = 56.15032650420169
Optimization restart 2/5, f = 75.18707283170784
Optimization restart 3/5, f = 75.1870728738366
Optimization restart 4/5, f = 56.15032650404075
Optimization restart 5/5, f = 56.150326504204685


GP_regression.,value,constraints,priors
rbf.variance,191.91473074701486,+ve,
rbf.lengthscale,42.643154754313514,+ve,
Gaussian_noise.variance,6.06424191323501,+ve,


#### Acquisition Function

In [9]:
model_variance = ModelVariance(model=emukit_model_ed)

#### Optimizer

In [10]:
optimizer = GradientAcquisitionOptimizer(parameter_space)


#### Experimental Design Loop

In [11]:
ed_loop = ExperimentalDesignLoop(
    model = emukit_model_ed,
    space = parameter_space,
    acquisition = model_variance,
    acquisition_optimizer = optimizer,
    batch_size = 1
)

In [12]:
ed_loop.run_loop(user_function_co2, 250)

Optimization restart 1/5, f = 56.15032650404075
Optimization restart 2/5, f = 75.1870728316659
Optimization restart 3/5, f = 75.18707282820958
Optimization restart 4/5, f = 75.18707275550769
Optimization restart 5/5, f = 56.150326504032705
[[ 3.   8.   5.  70.   3.   1.5]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 4.588696775183214

Optimization restart 1/5, f = 58.63880975839763
Optimization restart 2/5, f = 78.53612226895243
Optimization restart 3/5, f = 78.53612230360629
Optimization restart 4/5, f = 78.53612227215721
Optimization restart 5/5, f = 78.53612226237958
[[ 3.  8. 27. 70.  3.  5.]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 6.643277920100438

Optimization restart 1/5, f = 60.99248825243542
Optimization restart 2/5, f = 81.98485901529602
Optimization restart 3/5, f = 81.98485901538236
Optimization restart 4/5, f = 81.98485900739

 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square


Optimization restart 5/5, f = 86.62262421832045
[[ 3.         25.         27.         52.61793287  3.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 17.79653818840505

Optimization restart 1/5, f = 89.9601469187954
Optimization restart 2/5, f = 106.44472280709526
Optimization restart 3/5, f = 89.96014469716273
Optimization restart 4/5, f = 106.44472279878244
Optimization restart 5/5, f = 89.96014656468486
[[20.         14.84398571 27.         52.76809875  3.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 4.137216401285258

Optimization restart 1/5, f = 93.19818623729977
Optimization restart 2/5, f = 110.07138176726376
Optimization restart 3/5, f = 93.19818623716183
Optimization restart 4/5, f = 110.07138187827884
Optimization restart 5/5, f = 110.07138176820399
[[20.   8.   5.  70.   1.   1.5]]

User functi

 Retrying in 1 seconds

Output 8.82026362398641

Optimization restart 1/5, f = 147.0827721942586
Optimization restart 2/5, f = 147.08275346751367
Optimization restart 3/5, f = 180.35951812110764
Optimization restart 4/5, f = 180.35951811375193
Optimization restart 5/5, f = 147.0827534649789
[[ 3.  8. 27. 30.  1.  5.]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 24.068538394415356

Optimization restart 1/5, f = 149.63405195595433
Optimization restart 2/5, f = 185.4737084251595
Optimization restart 3/5, f = 149.633833279869
Optimization restart 4/5, f = 149.63383332034934
Optimization restart 5/5, f = 149.6338332790645
[[20.          8.          5.         55.77028829  3.          5.        ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 2.1163449318303273

Optimization restart 1/5, f = 152.166630842781
Optimization restart 2/5, f = 189.0126428226

 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


...

Optimization restart 2/5, f = 689.1611897416672
Optimization restart 3/5, f = 689.1611896992595
Optimization restart 4/5, f = 689.1611897012722
Optimization restart 5/5, f = 689.1611897051607
[[12.  25.   5.  70.   3.   1.5]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 1.964294436790003

Optimization restart 1/5, f = 691.0717791269539


 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 2/5, f = 691.071775500959
Optimization restart 3/5, f = 691.071775627152
Optimization restart 4/5, f = 690.8500784294039
Optimization restart 5/5, f = 691.0717755043313
[[ 8.         25.         19.85058242 49.28757589  1.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 11.72338875869855

Optimization restart 1/5, f = 692.7777120458333
Optimization restart 2/5, f = 693.1653142913503
Optimization restart 3/5, f = 693.1653142903322
Optimization restart 4/5, f = 997.0993529366374
Optimization restart 5/5, f = 997.0993564871542
[[20.         21.23698511 21.38217818 47.11401802  3.          5.        ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 7.9257990650620185

Optimization restart 1/5, f = 694.5433607698899
Optimization restart 2/5, f = 694.9103578412027
Optimization restart 3/5, f = 694.9103578401932
Opt

 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/paramz/transformations.py:111: RuntimeWarning:overflow encountered in expm1


Optimization restart 2/5, f = 704.2580691975602
Optimization restart 3/5, f = 704.2580692023545
Optimization restart 4/5, f = 704.2580692181349
Optimization restart 5/5, f = 704.2580691976694


### Experimental Design - Integrated Variance Reduction

#### Model / Emulator (GP)

In [13]:
emulator_ird = GPy.models.GPRegression(init_X, init_Y)
emukit_model_ird = GPyModelWrapper(emulator_ird, n_restarts=5)
emukit_model_ird.optimize()

Optimization restart 1/5, f = 56.15032650420169
Optimization restart 2/5, f = 75.18707283375363
Optimization restart 3/5, f = 75.18707287586867
Optimization restart 4/5, f = 75.18707286443053
Optimization restart 5/5, f = 75.1870728357923


#### Acquisition Function

In [14]:
integrated_variance_reduction = IntegratedVarianceReduction(
    model=emukit_model_ird, 
    space=parameter_space, 
    num_monte_carlo_points=100
)

#### Optimizer

In [15]:
optimizer = GradientAcquisitionOptimizer(parameter_space)

#### Experimental Design Loop

In [16]:
ed_loop = ExperimentalDesignLoop(
    model = emukit_model_ird,
    space = parameter_space,
    acquisition = integrated_variance_reduction,
    acquisition_optimizer = optimizer,
    batch_size = 1
)

In [17]:
ed_loop.run_loop(user_function_co2, 250)

Optimization restart 1/5, f = 56.15032650420169
Optimization restart 2/5, f = 56.15032652664891
Optimization restart 3/5, f = 75.18707282629595
Optimization restart 4/5, f = 56.15032650726235
Optimization restart 5/5, f = 56.1503265079982


 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square


[[ 3.         25.         27.         51.44303288  3.          2.18231638]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 21.892875892123094

Optimization restart 1/5, f = 60.04958545215801
Optimization restart 2/5, f = 80.54086184247355
Optimization restart 3/5, f = 80.5408617954191
Optimization restart 4/5, f = 80.54086176419364
Optimization restart 5/5, f = 60.049585450874645
[[ 3.         25.          5.         30.          2.          2.77881024]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 13.718965734441923

Optimization restart 1/5, f = 63.100755932033636
Optimization restart 2/5, f = 84.62291137243031
Optimization restart 3/5, f = 84.62291136671108
Optimization restart 4/5, f = 84.62291137238535
Optimization restart 5/5, f = 84.62291139167966
[[ 3.          8.44146971  5.         68.66956851  3.          3.56933202]]

User function time

 Retrying in 1 seconds

Output 6.122309641626681

Optimization restart 1/5, f = 114.79562847861459
Optimization restart 2/5, f = 114.79562847635398
Optimization restart 3/5, f = 114.79562847665146
Optimization restart 4/5, f = 114.79562848172874
Optimization restart 5/5, f = 114.79562847638618


 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square


...

Optimization restart 3/5, f = 629.4728062001703
Optimization restart 4/5, f = 629.4728062147176
Optimization restart 5/5, f = 629.4728062063921
[[16.         16.73732529 11.28668981 35.57162734  1.          3.610378  ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 7.811693018561621

Optimization restart 1/5, f = 631.1964079046078


 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square


Optimization restart 2/5, f = 631.1964075914275
Optimization restart 3/5, f = 1011.0465945267636
Optimization restart 4/5, f = 631.196407591416
Optimization restart 5/5, f = 631.1964075914602
[[ 7.         20.62425071 18.68545869 39.97526594  1.          1.5       ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 13.573335715360319

Optimization restart 1/5, f = 633.506795937058


 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square
 /Users/Matteo/opt/anaconda3/envs/ml_and_physical/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


Optimization restart 2/5, f = 700.3619411591567
Optimization restart 3/5, f = 709.844452891414
Optimization restart 4/5, f = 1015.0828135387922
Optimization restart 5/5, f = 633.5064294275643
[[ 3.         15.08871935 11.86685955 32.26616762  1.          5.        ]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 40.625285181946076

Optimization restart 1/5, f = 644.2027612690663
Optimization restart 2/5, f = 644.2027612711872
Optimization restart 3/5, f = 644.2027612688021
Optimization restart 4/5, f = 711.8626948081675
Optimization restart 5/5, f = 644.2027612700452
[[16.         12.06056212 16.8494527  42.19900559  3.          3.16201332]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds

Output 7.675087539303898

Optimization restart 1/5, f = 646.1824492996794
Optimization restart 2/5, f = 646.1821636092648
Optimization restart 3/5, f = 646.1821636092141
O

### Random GP

Supervised learning

In [18]:
emulator_rnd = GPy.models.GPRegression(train_X, train_Y)
emukit_model_rnd = GPyModelWrapper(emulator_rnd, n_restarts=50)
emukit_model_rnd.optimize()
emulator_rnd

Optimization restart 1/50, f = 539.8867979140487
Optimization restart 2/50, f = 539.886797914606
Optimization restart 3/50, f = 539.8867986444445
Optimization restart 4/50, f = 539.8867979140381
Optimization restart 5/50, f = 539.8867979140337
Optimization restart 6/50, f = 1011.5313294187904
Optimization restart 7/50, f = 539.8867979142575
Optimization restart 8/50, f = 539.8867979367768
Optimization restart 9/50, f = 1011.5313294186967
Optimization restart 10/50, f = 1011.5313293975598
Optimization restart 11/50, f = 640.5805051109465
Optimization restart 12/50, f = 1011.5313294203813
Optimization restart 13/50, f = 539.8867979145632
Optimization restart 14/50, f = 539.8867979152759
Optimization restart 15/50, f = 1011.5313294190926
Optimization restart 16/50, f = 539.886797914525
Optimization restart 17/50, f = 1011.5313293100812
Optimization restart 18/50, f = 539.886797926981
Optimization restart 19/50, f = 539.8867979140316
Optimization restart 20/50, f = 539.886797914105
Optimiz

GP_regression.,value,constraints,priors
rbf.variance,170.2890196036634,+ve,
rbf.lengthscale,19.184082679925318,+ve,
Gaussian_noise.variance,1.631542729336247,+ve,


### Evaluate

Evaluate RMSE of each model on our test set of 50 randomly selected points.

In [19]:
pred_Y, pred_Y_variance = emukit_model_ed.predict(test_X)
mse_ed = np.sqrt(np.mean((pred_Y-test_Y)**2))
print('Experimental Design w/ Model Variance RMSE: ', mse_ed)

Experimental Design w/ Model Variance RMSE:  1.147230139986604


In [20]:
pred_Y, pred_Y_variance = emukit_model_ird.predict(test_X)
mse_ird = np.sqrt(np.mean((pred_Y-test_Y)**2))
print('Experimental Design w/ Integrated Variance Reduction RMSE: ', mse_ird)

Experimental Design w/ Integrated Variance Reduction RMSE:  1.1070473123093656


In [21]:
pred_Y, pred_Y_variance = emukit_model_rnd.predict(test_X)
mse_rnd = np.sqrt(np.mean((pred_Y-test_Y)**2))
print('Random GP RMSE: ', mse_rnd)

Random GP RMSE:  1.394084396039147


In [22]:
p = 1 - (mse_ed/mse_rnd)
f'Experimental Design w/ Model Variance is {p*100:.3f}% better than training on random points.'

'Experimental Design w/ Model Variance is 17.707% better than training on random points.'

In [23]:
p = 1 - (mse_ird/mse_rnd)
f'Experimental Design w/ Integrated Variance Reduction is {p*100:.3f}% better than training on random points.'

'Experimental Design w/ Integrated Variance Reduction is 20.590% better than training on random points.'

### Save Models

In [24]:
with open(f'experimental_design_new_outputs/models/model_variance_20_init_points_250_loops_co2.pkl', "wb") as f:
     pickle.dump(emukit_model_ed, f)

with open(f'experimental_design_new_outputs/models/integrated_variance_reduction_20_init_points_250_loops_co2.pkl', "wb") as f:
     pickle.dump(emukit_model_ird, f)
        
with open(f'experimental_design_new_outputs/models/270_random_points_supervised_GP_co2.pkl', "wb") as f:
     pickle.dump(emukit_model_rnd, f)